In [4]:
import libsvm.svmutil as svm
import numpy as np

c:\MiniConda\lib\site-packages\libsvm\svm.py:147: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def csr_to_problem_jit(l, x_val, x_ind, x_rowptr, prob_val, prob_ind, prob_rowptr, indx_start):


In [5]:
y_train, x_train = svm.svm_read_problem("train.txt",return_scipy=True)
y_train = y_train.reshape(-1,1)
x_train = x_train.toarray()
train_data = np.concatenate((y_train,x_train),axis=1)
y_test, x_test = svm.svm_read_problem("test.txt",return_scipy=True)
y_test = y_test.reshape(-1,1)
x_test = x_test.toarray()
test_data = np.concatenate((y_test,x_test),axis=1)

In [25]:
train_data[:3]

array([[ 7.    ,  3.    ,  0.285 ,  0.225 ,  0.07  ,  0.1005,  0.0425,
         0.0185,  0.035 ],
       [10.    ,  1.    ,  0.62  ,  0.48  ,  0.165 ,  1.0855,  0.481 ,
         0.2575,  0.305 ],
       [12.    ,  2.    ,  0.415 ,  0.325 ,  0.105 ,  0.38  ,  0.1595,
         0.0785,  0.12  ]])

In [6]:
class Node():
    def __init__(self) -> None:
        self.left = None
        self.right = None
        self.parent = None
        
        self.best_dim = None
        self.best_theta = None
        self.best_y = None
        self.is_leaf = False
    def updateData(self,best_dim,best_s,best_theta):
        self.best_dim = best_dim
        self.best_s = best_s
        self.best_theta = best_theta

In [7]:
def check_all_same_y(y_array:np.array):
    N = y_array.shape
    y_1 = y_array[0]
    y_1_array = np.ones(N) * y_1
    # print(y_array,y_1_array)
    return np.all(y_1_array == y_array)

In [41]:
a = np.array([1,1,1,1,2])
# check_y(a)

[1 1 1 1 2] [1. 1. 1. 1. 1.]


False

In [8]:
def check_all_same_x(x_array:np.array):
    N, dim = x_array.shape
    x_1 = x_array[0]
    x_1_array = np.ones(N*dim).reshape(N,dim) * x_1
    return np.all(x_1_array == x_array)

In [55]:
# a = np.ones(6).reshape(2,3)
b = np.arange(6).reshape(2,3)
c = np.array([[1,2,3],[1,2,3]])
# a * b
# check_x(b)

False

In [68]:
a = np.array([2,9,5,4])
b = np.array([3,1,6,8])
c = np.column_stack((a,b))
c

array([[2, 3],
       [9, 1],
       [5, 6],
       [4, 8]])

In [72]:
np.sort(c,axis=0)

array([[2, 1],
       [4, 3],
       [5, 6],
       [9, 8]])

In [71]:
c[np.argsort(c,axis=0)[:,0]]

array([[2, 3],
       [4, 8],
       [5, 6],
       [9, 1]])

In [75]:
(2 + -np.inf) / 2

-inf

In [82]:
a = np.arange(6).reshape(2,3)
np.unravel_index(a.argmin(),a.shape)

(0, 0)

In [93]:
a = np.array([1,3,3,3,2,1])
b = np.array([1,3,3,4,2,1])
b[int(np.median([np.arange(b.size)[b == b.max()]]))]

4

In [9]:
def decisionStumpRegression(data:np.array):
    # data : (x,y)
    N, _ = data.shape
    data = data[np.argsort(data,axis=0)[:,0]] # sort along x value
    impurity = np.zeros(N)
    for index in range(1,N):
        left_impurity = np.mean(np.square(data[:index,1] - np.mean(data[:index,1])))
        right_impurity = np.mean(np.square(data[index:,1] - np.mean(data[index:,1])))
        # left_size = index, right_size = N - index
        impurity[index] = left_impurity * index + right_impurity * (N - index)
    impurity[0] = np.sum(np.square(data[:,1] - np.mean(data[:,1])))
    best_index = int(np.median(np.arange(N)[impurity == impurity.min()]))
    # 0:best_index is left part, best_index:last is right part
    if best_index == 0:
        best_theta =  -np.inf
    else:
        best_theta = (data[best_index - 1,0] + data[best_index,0]) / 2
    return best_index,impurity[best_index],best_theta

In [10]:
def updateBestParameter(root:Node,data:np.array):
    N, dim = data.shape
    best_impurity = np.inf
    for d in range(dim-1,0,-1): # dim includes y, so actually dim is dim + 1
        temp_index,temp_impurity,temp_theta = decisionStumpRegression(np.column_stack((data[:,d],data[:,0]))) # (x,y)
        if temp_impurity <= best_impurity:
            best_index = temp_index
            best_impurity = temp_impurity
            best_theta = temp_theta
            best_dim = d
    
    root.best_dim = best_dim
    root.best_theta = best_theta
    root.best_y = -1

    return best_index

In [100]:
np.arange(10,0,-1)

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1])

In [11]:
def CART(data:np.array,root:Node):
    check_y_result = check_all_same_y(data[:,0])
    if check_y_result == True:
        root.is_leaf = True
        root.best_y = data[0][0]
        return
    check_x_result = check_all_same_x(data[:,1:])
    if check_x_result == True:
        root.is_leaf = True
        root.best_y = np.mean(data[:,0])
        return
    best_index = updateBestParameter(root,data)
    
    if best_index == 0:
        root.best_y = np.mean(data[:,0])
        root.is_leaf = True
    else:
        # sort along x value
        left_part = data[np.argsort(data,axis=0)[:,root.best_dim]][:best_index]
        right_part = data[np.argsort(data,axis=0)[:,root.best_dim]][best_index:]
        
        root.left = Node()
        root.left.parent = root
        
        root.right = Node()
        root.right.parent = root
        
        
        CART(left_part,root.left)
        CART(right_part,root.right)
    

In [12]:
root = Node()

In [20]:
CART(train_data,root) 

In [22]:
from io import TextIOWrapper
def printTree(root:Node,level:int,file:TextIOWrapper):
    if root != None:
        printTree(root.left,level+1,file)
        print("\t" * level,end="")
        file.write("\t" * level)
        if root.is_leaf == True:
            print(f"B_y:{root.best_y:.3f}, level:{level:2d}")
            file.write(f"B_y:{root.best_y:.3f}, level:{level:2d}\n")
        else:
            print(f"B_dim:{root.best_dim}, B_theta:{root.best_theta:.3f}, level:{level:2d}")
            file.write(f"B_dim: {root.best_dim}, B_theta: {root.best_theta:.3f}, level:{level:2d}\n")
        printTree(root.right,level+1,file)

In [25]:
# F = open("result.txt",'w')
# printTree(root,0,F)
# F.close()

In [29]:
a = np.array([[1,2],[3,4]])
b = np.array([[1,2],[3,4]])
np.row_stack((a,b))

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

In [28]:
b[0] = 10
a

array([1, 2, 3, 4, 5])

In [43]:
def CARTprediction(root:Node,data:np.array):
    if root != None:
        if root.is_leaf == True:
            data[:,-1] = root.best_y
            return data[:,-2:]
        else:
            best_dim = root.best_dim
            best_theta = root.best_theta
            
            left = data[data[:,best_dim] < best_theta]
            right = data[data[:,best_dim] >= best_theta]
            if left.size != 0 and right.size != 0: 
                y_pred_left = CARTprediction(root.left,left)
                y_pred_right = CARTprediction(root.right,right)
                return np.row_stack((y_pred_left,y_pred_right))
            elif left.size != 0:
                return CARTprediction(root.left,left)
            elif right.size != 0:
                return CARTprediction(root.right,right)

In [31]:
size = test_data.shape[0]
test_data = np.column_stack((test_data,np.arange(size),np.zeros(size)))
test_data[0]

array([18.    ,  1.    ,  0.57  ,  0.475 ,  0.195 ,  1.0295,  0.4635,
        0.1905,  0.305 ,  0.    ,  0.    ])

array([[18.    ,  1.    ,  0.57  ,  0.475 ,  0.195 ,  1.0295,  0.4635,
         0.1905,  0.305 ,  0.    ,  0.    ]])

In [56]:
y_pred = CARTprediction(root,test_data)

In [57]:
y_pred = y_pred[np.argsort(y_pred,axis=0)[:,0]]

In [62]:
np.mean(np.square(y_pred[:,1] - test_data[:,0]))

9.654042988741043

In [97]:
a = np.arange(20).reshape(10,2)
sam = np.random.randint(0,10.0,10)
max(err)

18.452405

In [101]:
import matplotlib.pyplot as plt
import numpy as np

err = np.genfromtxt("each_err.txt")
plt.figure(figsize=(10,5))
plt.hist(err,bins=np.arange(7,20,1))
plt.xticks(np.arange(7,20,1))
plt.yticks(np.arange(100,1100,100))
plt.grid()
plt.savefig("Q11_plot")
plt.close()
plt.figure(figsize=(10,10))
avg_err = np.genfromtxt("avg_err.txt")
plt.plot(avg_err)
plt.yticks(np.arange(9.5,12.25,0.25))
plt.savefig("Q12_plot")
plt.close()
